In [1]:
USE_SAME_FEW_SHOTS=True

# INSTALLAZIONE E IMPORT LIBRERIE

In [7]:
import glob
import pandas as pd
import pickle

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
from google.colab import drive
from IPython.display import Image, display

mount_point = "/content/gdrive"
base_path = mount_point + "/MyDrive/Colab/SDG/data"
# input_data_path = base_path + "/input/sdg_17_labels_classification_dataset_1020_texts_TEST.xlsx"
# input_data_path_train = base_path + "/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN.xlsx"
input_data_path = base_path + "/input/sdg_17_labels_classification_dataset_1020_texts_TEST_2023.12.11.xlsx"
input_data_path_train = base_path + "/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN_2023.12.11.xlsx"


output_data_path = base_path + "/../Llama-2/data/output/2023.11.21-2_LLama2_7B-chat-hf_4-bit_float-16_5-few-shot"

drive.mount(mount_point, force_remount=True)
# data_path = '/content/gdrive/MyDrive/Colab/SDG/Data/input/2023.10.17-1_GPT_zero-shot-learning_all-17-sdg.xlsx'

Mounted at /content/gdrive


In [4]:
model_id = "meta-llama/Llama-2-7b-chat-hf"
# model_id = "meta-llama/Llama-2-13b-chat-hf"

In [5]:
nbit = 4
llamasize = '7B'
# llamasize = '13B'
temp = 0.1
# outdir = 'data/output/2023.10.20-2_LLama2_langchain/'
# outdir = 'data/output/2023.11.21-1_LLama2_7B-chat-hf_4-bit_float-16_3-few-shot/'
# outfilename = outdir+f'sdg_classification_llama_1020-texts_temp-{temp}_nbit-{nbit}_llamasize-{llamasize}.xlsx'
# outfilename

In [7]:
#!pip install --upgrade pip
!pip install transformers # Download e caricamento modelli
!pip install bitsandbytes # Quantizzazione
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 3.5 MB/s eta 0:00:00


In [8]:
# Installazione "permanente"
!pip install sentencepiece && touch ~/.ipython/profile_default/startup/00-first.py && echo "import <sentencepiece>" >> ~/.ipython/profile_default/startup/00-first.py

In [9]:
!pip install pillow==10.0.1
!pip install langchain[llms]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 31.3 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.9/815.9 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.0/631.0 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [10]:
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate

from langchain.llms import HuggingFacePipeline
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate

In [11]:
import transformers
from transformers import LlamaForCausalLM, LlamaTokenizer, GenerationConfig # Questi moduli sono dedicati al modello LLama2
from transformers import AutoModelForCausalLM, AutoTokenizer # Questi moduli permettono di scaricare qualsiasi modello e tokenizer
import pandas as pd
import torch # PyTorch
import langchain

In [12]:
torch.cuda.empty_cache()

# LOGIN DA HUGGING FACE

In [13]:
from IPython.display import clear_output
huggingface_cli_token = input("INSERT YOUR huggingface-cli_token:")
clear_output()

In [14]:
!huggingface-cli login --token $huggingface_cli_token

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# DOWNLOAD E CARICAMENTO DEL MODELLO

In [15]:
# pip install --upgrade bitsandbytes

In [16]:
model = LlamaForCausalLM.from_pretrained(  # Caricamento del modello usando la libreria dedicata. In questo caso, "ForCausal" si riferisce al modello per la generazione di testo
    model_id, # Nome del modello precedentemente impostato
    load_in_4bit = True, # Quantizzazione in 4 bit (per 8 bit usare il parametro load_in_8bit)
    torch_dtype = torch.float16 # Impostare questa codifica aiuta il modello in fase di generazione, in quanto il prompt è tipicamente float16
)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [17]:
torch.cuda.empty_cache()

In [18]:
!nvidia-smi

Fri Feb 16 17:41:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0              27W /  70W |   4333MiB / 15360MiB |      3%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [19]:
tokenizer = LlamaTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

In [20]:
torch.cuda.empty_cache()

In [21]:
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm

In [22]:
torch.cuda.empty_cache()

## TEST

In [23]:
generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    temperature=0.1,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=200,  # max number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

In [24]:
torch.cuda.empty_cache()

In [25]:
llm = HuggingFacePipeline(pipeline=generate_text)

In [26]:
torch.cuda.empty_cache()

In [16]:
# data = pd.read_excel("data/input/data_sel_4_few_shot_examples.xlsx")
# data = pd.read_excel("data/input/2023.10.17-1_GPT_zero-shot-learning_all-17-sdg.xlsx")
# data = pd.read_excel("../data/input/2023.11.21-1_LLama2_7B-chat-hf_4-bit_float-16_3-few-shot/sdg_17_labels_classification_dataset_1020_texts_TEST.xlsx")
data = pd.read_excel(input_data_path)
# data = data.drop(columns=['Unnamed: 0', 'goal_label', 'motivation_for_the_answer', 'goal_label_int', 'sdg_probability'])
data

,Unnamed: 0.1,Unnamed: 0,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length
0,0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38
1,1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15
2,2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125
3,3,3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,146
4,4,4,2143,NaN,By Dipo Olowookere. The President of the Manuf...,0,NaN,NaN,NaN,124
...,...,...,...,...,...,...,...,...,...,...
1015,1015,1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,100
1016,1016,1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,60
1017,1017,1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,124
1018,1018,1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,109


In [28]:
data.shape

(1020, 10)

In [29]:
data.groupby('sdg').count()['text']

sdg
0     60
1     60
2     60
3     60
4     60
5     60
6     60
7     60
8     60
9     60
10    60
11    60
12    60
13    60
14    60
15    60
16    60
Name: text, dtype: int64

In [17]:
data_sel = data

In [31]:
if USE_SAME_FEW_SHOTS==True:
    data_sel_4_few_shot_examples = pd.read_excel(input_data_path_train).drop(columns=['Unnamed: 0'])
    data_sel_4_few_shot_examples = data_sel_4_few_shot_examples[
        (data_sel_4_few_shot_examples.text_id==5614) |
        (data_sel_4_few_shot_examples.text_id==2089) |
        (data_sel_4_few_shot_examples.text_id==5665) |
        (data_sel_4_few_shot_examples.text_id==370)  |
        (data_sel_4_few_shot_examples.text_id==1422)
    ]
else:
    data_sel_4_few_shot_examples = pd.read_excel(input_data_path_train).drop(columns=['Unnamed: 0']).sample(5)

data_sel_4_few_shot_examples

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length
738,738,370,10.1787/9789264272392-3-en,"For example, the poorest 20% of farm household...",2,0.0,9.0,1.000000,96
1092,1092,1422,10.1787/9789264283299-en,"Nevertheless, the average number of physician ...",3,1.0,8.0,0.777778,76
2319,2319,5665,10.18356/3fe10a08-en,"As a result, both the trade and current accoun...",8,0.0,5.0,1.000000,53
3569,3569,2089,10.18356/ee52a573-en,"This refers to the manufacture, use and dispos...",12,3.0,29.0,0.812500,47
4608,4608,5614,10.2139/SSRN.1406042,The modern definitions of the inchoate offence...,16,0.0,6.0,1.000000,136


In [32]:
few_shots = ""

for _, row in data_sel_4_few_shot_examples[['text', 'sdg']].iterrows():
    input_text = "Input text: \""+row['text']+"\"\n"
    output_text = "Output: \"---SDG "+str(row['sdg'])+'---\"\n\n'

    few_shots += input_text
    few_shots += output_text

print(few_shots)

Input text: "For example, the poorest 20% of farm households in the Philippines tend to be small-scale producers and landless workers whose rice production does not satisfy their own needs and who therefore purchase 77% of the rice that they consume on average (Chapter 4). Regional levels of stunting among children under five years of age, for example, remain worryingly high (Figure 3). While economic growth has enabled significant declines in populations suffering from chronic poverty, for some households, it may have only altered the nature of food insecurity from permanent to transitory, rather than eliminate it entirely."
Output: "---SDG 2---"

Input text: "Nevertheless, the average number of physician contacts per person is not greater than the EU average. For example, home visits to patients by GPs are a regular practice and patients do not usually have to wait long to get access to GPs, although waiting times for more specialised services (e.g. mental health specialists, ophthal

In [33]:
template = """[INST]
<<SYS>>
Classify the following text between triple quotes according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015, choosing one (and only one) of the SDG labels based on it best representing the content of the entire text enclosed in triple quotes.

In the following lines, a text enclosed in triple quotes will be given to you.

The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "SDG 0".

The following are 5 examples of expected classification given an input text:

"""+few_shots+"""


The text to be processed is:
<</SYS>>

'''
{text}
'''

<<SYS>>
Answer with label only usign the format:---LABEL---.
<</SYS>>
[/INST]
"""

In [34]:
prompt = PromptTemplate(
    input_variables=["text"],
    template=template,
)

In [35]:
torch.cuda.empty_cache()

In [36]:
%%time
#results = data_sel.apply(lambda x: llm(prompt.format(text=x["text"])), axis=1)

cnt = 0
results = []
for index, row in data_sel.iterrows():

    text_id = row['text_id']
    text = row["text"]
    sdg_true = row["sdg"]

    try:
      result = llm(prompt.format(text=text))
    except Exception as e:
      print(cnt, text_id, sdg_true, e)

    result_extended = (cnt, text_id, sdg_true, result)

    print(result_extended)

    with open(output_data_path+f'/sdg_classification_Llama2_7B_{cnt}.pkl', 'wb') as f:
        pickle.dump(result_extended, f)

    results.append(result_extended)

    cnt+=1
    #time.sleep(0.5)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


(0, 6668, 0, '---SDG 9---')
(1, 6099, 0, '---SDG 8---')
(2, 6471, 0, '---SDG 16---')
(3, 1786, 0, '---SDG 4---')
(4, 2143, 0, "---SDG 17---\n\nThe text you provided can be classified under SDG 17 - Strategic Partnerships for Sustainable Development. The President of the Manufacturers Association of Nigeria (MAN) is questioning the pricing of gas generated for electricity in dollars rather than naira, the nation's legal tender. This highlights the issue of uneven exchange rates and the impact it has on the economy, particularly for small businesses and citizens. It also shows the need for strategic partnerships to address these issues and find sustainable solutions.")
(5, 3377, 0, '---SDG 16---')
(6, 5589, 0, '---SDG 16---')
(7, 5431, 0, '---SDG 8---')
(8, 768, 0, '---SDG 9---')
(9, 2735, 0, '---SDG 9---')
(10, 1306, 0, '---SDG 16---')
(11, 3844, 0, '---SDG 13---')
(12, 1524, 0, '---SDG 13---')
(13, 4660, 0, '---SDG 3---')
(14, 5077, 0, '---SDG 16---')
(15, 995, 0, '---SDG 11---')
(16, 

In [8]:
files = glob.glob(output_data_path + "/*.pkl")

In [9]:
results_complete = []
for file in files:
    res = pd.read_pickle(file)

    # try:
    #     if len(res[3]['choices'])>1:
    #         raise Exception('ERROR: Multilabel detected!')
    #         break
    #     sdg_pred = res[3]['choices'][0]['message']['content']
    #     res4df = (res[0], res[1], res[2], sdg_pred)
    #     print(res4df)
    #     tuple_list.append(res4df)
    # except:
    #     print("#################################################################")
    #     print(res)
    #     print("#################################################################")

    # print(res)
    results_complete.append(res)
    # break

In [11]:
results_complete[:5]

[(20, 1376, 0, '---SDG 13---'),
 (21, 3889, 0, '---SDG 13---'),
 (22, 6576, 0, '---SDG 3---'),
 (23, 3912, 0, '---SDG 13---'),
 (24, 1565, 0, '---SDG 12---')]

In [12]:
results = results_complete

In [19]:
results_processed = pd.DataFrame(results, columns=['cnt', 'text_id', 'sdg_true', 'result'])
results_processed

,cnt,text_id,sdg_true,result
0,20,1376,0,---SDG 13---
1,21,3889,0,---SDG 13---
2,22,6576,0,---SDG 3---
3,23,3912,0,---SDG 13---
4,24,1565,0,---SDG 12---
...,...,...,...,...
1015,15,995,0,---SDG 11---
1016,16,2166,0,---SDG 1---
1017,17,2512,0,---SDG 16---
1018,18,2191,0,---SDG 3---


In [20]:
results_processed['label'] = results_processed['result'].str.extract(r'SDG (\d+)')
results_processed

,cnt,text_id,sdg_true,result,label
0,20,1376,0,---SDG 13---,13
1,21,3889,0,---SDG 13---,13
2,22,6576,0,---SDG 3---,3
3,23,3912,0,---SDG 13---,13
4,24,1565,0,---SDG 12---,12
...,...,...,...,...,...
1015,15,995,0,---SDG 11---,11
1016,16,2166,0,---SDG 1---,1
1017,17,2512,0,---SDG 16---,16
1018,18,2191,0,---SDG 3---,3


In [21]:
final = data_sel.merge(results_processed, left_on='text_id', right_on='text_id')
final

,Unnamed: 0.1,Unnamed: 0,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,sdg_true,result,label
0,0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38,0,0,---SDG 9---,9
1,1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15,1,0,---SDG 8---,8
2,2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125,2,0,---SDG 16---,16
3,3,3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,146,3,0,---SDG 4---,4
4,4,4,2143,NaN,By Dipo Olowookere. The President of the Manuf...,0,NaN,NaN,NaN,124,4,0,---SDG 17---\n\nThe text you provided can be c...,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,1015,1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,100,1015,16,---SDG 16---,16
1016,1016,1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,60,1016,16,---SDG 16---,16
1017,1017,1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,124,1017,16,---SDG 16---,16
1018,1018,1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,109,1018,16,---SDG 16---,16


In [22]:
# final.to_excel(outfilename)
final.to_excel(output_data_path+"/2023.11.21-2_LLama2_7B-chat-hf_4-bit_float-16_5-few-shot.xlsx")

In [23]:
import numpy as np
from sklearn.metrics import classification_report

y_true = final.sdg
y_pred = final.label
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1020
1020
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        60
           1       0.50      0.77      0.61        60
           2       0.93      0.23      0.37        60
           3       0.74      0.75      0.74        60
           4       0.65      0.37      0.47        60
           5       0.94      0.48      0.64        60
           6       0.86      0.62      0.72        60
           7       1.00      0.35      0.52        60
           8       0.40      0.42      0.41        60
           9       0.44      0.60      0.51        60
          10       0.43      0.35      0.39        60
          11       0.78      0.30      0.43        60
          12       0.61      0.77      0.68        60
          13       0.38      0.67      0.49        60
          14       0.96      0.45      0.61        60
          15       0.60      0.85      0.70        60
          16       0.31      0.88      0.46        60
          17     

In [ ]:
# classification_report_filename = output_data_path+f'/sdg_classification_llama_1020-texts_temp-{temp}_nbit-{nbit}_llamasize-{llamasize}_classification_report.xlsx'
# classification_report_filename

In [ ]:
# report=classification_report(y_true, y_pred, output_dict=True)
# report_df = pd.DataFrame(report).transpose()
# report_df.to_excel(classification_report_filename)

In [ ]:
# test = pd.read_excel(classification_report_filename)
# test

In [24]:
final_new = final

In [33]:
results_processed_merged = final_new
results_processed_merged = results_processed_merged.rename(columns={'label': 'predicted_label_clean'})

In [35]:
# CORREZIONI NEL CASO IN CUI IL MODELLO RESTITUISCA OUTPUT NON UTILI PER DEFINIRE LA CLASSE O CLASSI NON PREVISTE

# NEL CASO IN CUI IL MODELLO RESTITUISCA OUTPUT NON UTILI PER DEFINIRE LA CLASSE SI APPLICA LA CLASSE 0
results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'predicted_label_clean'] = 0
# df.loc[df['label'].isna(), 'label'] = df.loc[df['label'].isna(), 'sdg'] + 1

# NEL CASO IN CUI IL MODELLO RESTITUISCA CLASSI NON PREVISTE SI APPLICA LA CLASSE 0
results_processed_merged['predicted_label_clean'] = results_processed_merged.predicted_label_clean.apply(lambda x: 0 if int(x)>16 else x)

In [36]:
import numpy as np
from sklearn.metrics import classification_report

y_true = results_processed_merged.sdg
y_pred = results_processed_merged.predicted_label_clean
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1020
1020
              precision    recall  f1-score   support

           0       0.05      0.03      0.04        60
           1       0.50      0.77      0.61        60
           2       0.93      0.23      0.37        60
           3       0.74      0.75      0.74        60
           4       0.65      0.37      0.47        60
           5       0.94      0.48      0.64        60
           6       0.86      0.62      0.72        60
           7       1.00      0.35      0.52        60
           8       0.40      0.42      0.41        60
           9       0.44      0.60      0.51        60
          10       0.43      0.35      0.39        60
          11       0.78      0.30      0.43        60
          12       0.61      0.77      0.68        60
          13       0.38      0.67      0.49        60
          14       0.96      0.45      0.61        60
          15       0.60      0.85      0.70        60
          16       0.31      0.88      0.46        60

    accuracy    

In [37]:
report=classification_report(y_true, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df[['precision', 'recall', 'f1-score']] *= 100

In [38]:
accuracy = report_df.loc['accuracy']['precision']
precision = report_df.loc['macro avg']['precision']
recall = report_df.loc['macro avg']['recall']
f1_score = report_df.loc['macro avg']['f1-score']

In [39]:
print(precision, recall, accuracy, f1_score)

62.201789665536744 52.25490196078431 52.254901960784316 51.647747693403964


In [40]:
print(round(precision, 1), round(recall, 1), round(accuracy, 1), round(f1_score, 1))

62.2 52.3 52.3 51.6
